#Trimming

To analyze the data we will be using metaBEAT, a tool tailored towards reproducible and efficient analyses of metabarcoding data that was developed by Christoph Hahn at University of Hull. The pipeline is still under active development and will likely be extended further in the future. The pipeline is available in a Docker container with all necessary dependencies. The Docker image is building on ReproPhylo.

In this notebook we will be looking at C01 DNA sequences amplified from freshwater eDNA water collected from rivers across the UK. These samples are taken from sites with invasive alien species present and form part of 2 seperate experiments: 1 method comparrison (comparing kicknet sampling and eDNA metabarcoding, species specific approaches) to detect key inasive alien species and 2 detection of key invasive alien species downstream of known populations, to investigate transport of DNA in flowing water.

The metaBEAT tool is designed as a wrapper around a complete analysis from raw data, performing (optionally) de-multiplexing, quality filtering, clustering along the way, to OTU tables in biom format. It currently supports BLAST, Kraken and phylogenetic placement (pplacer). Further approaches will be included in the future to allow for efficient and standardized comparative assessments of all approaches.

metaBEAT offers a large number of options. 


The first step will be to trim/clean our raw Illumina data.

##Data input

This notebook will perform basic processing (read trimming, -merging-, chimera removal) of the eDNA data. Clustering and taxonomic assignment will be performed in a separate notebook.

Minimum input for an analysis is a set of query sequences in one or several files (accepted are a number of file formats, e.g. fasta, fastq). These will be run through the pipeline sequentially.
Information on the nature and location of the query sequence files must be provided in a separate tab-delimited text file via the -Q flags.

Each line in this text file should look as follows: unique sample_ID format file1 file2
The required text files can be generated in any text editor. So theoretically, nano could be used in the terminal to construct the text file. For reproducibility and ease, a simple program can be used to generate the required file.

In the cell below, it is produced using a simple python script. The script will list all files in the location to which you've downloaded your Illumina data (specified via the 'datadir' variable. It assumes that there is a file ending in _R1_001.fastq for each sample. For each such file, it will extract the sample name from the filename and format the required line for the text file accordingly. The resulting file is called Querymap.txt (specified in the 'to' variable).

In [ ]:
!mkdir trimming

In [ ]:
cd trimming/

In [ ]:
!ls -1 ../demultiplex/

In [ ]:
!echo "SampleID" > ../trimming/Sample_names.tsv

In [ ]:
%%bash

for a in $(ls ../demultiplex/ | grep "R1" | cut -d '.' -f 1)
do 
   SampleID=$a
   
   echo -e "$SampleID"
done >> ../trimming/Sample_names.tsv

In [ ]:
!cat ../trimming/Sample_names.tsv

Prepare a text file specifying the samples to be processed including the format and location of the reads.
The below command expects the Illumina data to be present in 2 fastq files (forward and reverse reads) per sample in a directory ./raw_reads/. It expects the files to be named 'sampleID-marker', followed by '_R1_001' or '_R2_001' to identify the forward/reverse read file respectively. sampleID must correspond to the first column in the file Sample_accessions.tsv, marker is 'C01'.

Read file names, for example:

../raw_reads/samplename-C01-miseqinformation_R1_001.fastq.gz

In [ ]:
%%bash

for a in $(cat ../trimming/Sample_names.tsv | grep "SampleID" -v)
do
    R1=$(ls -1 ../demultiplex/$a* | grep ".R1")
    R2=$(ls -1  ../demultiplex/$a* | grep ".R2")

    echo -e "$a\tfastq\t$R1\t$R2"
done > Querymap.txt

In [ ]:
!head Querymap.txt

In [ ]:
!head Querymap_global.txt

##Raw read processing

Now, perform basic quality trimming and clipping (Trimmomatic) and paired-end read merging (flash). metaBEAT will be used to process all 192 samples in one go.

In [ ]:
!metaBEAT_global.py -h

The amplicon is expected to be 313 bp long. With a readlength of 300 bp we don't expect to see any primer sequences, so it's not necessary to provide the primer sequence for the trimming algorithm.

Command to trim, remove adapter sequences and merge reads using the metaBEAT pipeline.

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap_global.txt \
--trim_qual 30 \
--merge --forward_only --product_length 313 \
-@ M.Benucci@2015.hull.ac.uk \
-n 5 -v &> log

In [ ]:
!tail -n 100 log

Read processing will take several hours.

##Visualise query survival after trimming

metaBEAT will generate a directory with all temporary files that were created during the processing for each sample and will record useful stats summarizing the data processing in the file metaBEAT_read_stats.csv. Should look roughly like this.

Can explore the table manually or quickly plot out some of these stats here:

In [ ]:
%matplotlib inline
import pandas as pd

df = pd.read_csv('metaBEAT_read_stats.csv',index_col=0)
df['fraction'] = df['queries']/(df['total']*0.5)
df.fraction.hist(bins=50)

The final step in the processing will be global clustering of the centroids from all clusters from all samples to produce denovo OTUs. The temporary files from the global clustering and the final OTU table were written to the directory GLOBAL.

In [ ]:
!ls GLOBAL/

The denovo OTU table (numbers are reads) can be briefly to see how OTUs are distributed across your samples.
Detailed information on what metaBEAT did to each sample is contained in the log file log. It contains the exact commands that were run for each sample during each step of the process.

It's a large text file - look at the first 100 lines.

## Chimera detection

Some stats on the read counts before/after trimming, merging etc. are summarized for you in read_stats.csv.

Next stage of the processing is chimera detection and removal of putative chimeric sequences. We'll do that using uchime as implemented in vsearch.

In [ ]:
cd ../

In [ ]:
!mkdir chimera_detection

In [ ]:
cd chimera_detection

Load file in fasta format to be used in chimera detection.

Prepare Refmap file, i.e. text file that specifies the location and the format of the reference to be used.
The reference sequences for freshwater macroinvertebrates are provided in a fasta file.

In [ ]:
!ls ../../reference_database/CO1_refdb/

In [ ]:
#!echo '../ecoPCR_Invert_DB.fasta\tfasta\n' > REFmap.txt

In [ ]:
%%bash

#Write REFmap
for file in $(ls ../../reference_database/CO1_refdb/* | grep "_SATIVA_cleaned.gb$")
do
    echo -e "$file\tgb"
done > REFmap.txt

In [ ]:
!head REFmap.txt

In [ ]:
!metaBEAT_global.py -h

In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ M.Benucci@2015.hull.ac.uk

In [ ]:
!head refs.fasta

Now run chimera detection.

In [ ]:
%%bash


for a in $(cut -f 1 ../trimming/Querymap_global.txt)
do
    if [ -s ../trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log 
        cd ..

    else
        echo -e "$a is empty"
    fi
done

In [ ]:
!tail Dv1_gut12_1/log